In [28]:
# This is a cell to hide code snippets from displaying
# This must be at first cell!

from IPython.display import HTML

hide_me = ''
HTML('''<script>
code_show=true; 
function code_toggle() {
  if (code_show) {
    $('div.input').each(function(id) {
      el = $(this).find('.cm-variable:first');
      if (id == 0 || el.text() == 'hide_me') {
        $(this).hide();
      }
    });
    $('div.output_prompt').css('opacity', 0);
  } else {
    $('div.input').each(function(id) {
      $(this).show();
    });
    $('div.output_prompt').css('opacity', 1);
  }
  code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input style="opacity:0" type="submit" value="Click here to toggle on/off the raw code."></form>''')

# The Battle of Neighborhoods
## Toronto neighborhood - Vegan restaurants

**Author**: Hércules Sant'Ana da Silva José

_Report created in July/2020_

## Summary

**Introduction Section:**
- Introduction where we discuss the business problem about vegan restaurants in Toronto and who would be interested in this project.

**Data Section:**
- Data where we describe the data that will be used to solve the problem and the source of the data.

**Methodology section:**
- Methodology section which represents the main component of this report where we discuss and describe any exploratory data analysis that we did, any inferential statistical testing that we performed, if any, and what machine learnings were used and why.

**Results section:**
- Results section where we discuss the results.

**Discussion section:**
- Discussion section where we discuss any observations you noted and any recommendations to our audience based on the results.

**Conclusion section:**
- Conclusion section where we conclude the report.

## 1. Introduction

### 1.1. Scenario

Recent studies suggest that about 8% of the world's population is vegan. Few people around the world are committed to living a plant-based lifestyle. If you are interested in eating more cleanly, do not want to explore animals or simply enjoy how you feel as a vegan, it can sometimes be difficult to find great vegan restaurants in the world.

However, this is starting to change. You will find that the main cities in the world are slowly but surely developing strong vegan cultures, from coffee shops and cooperatives to some of the best vegan restaurants in the world. In short, you now have more options than simply ordering the only salad on the menu.

Toronto is one of the most ethnic and culturally diverse cities in the world. Located on the shores of Lake Ontario, it has people from all over the world from different cultures and ethnicities resulting in a lot of delicious delights. No matter what type of cuisine you like, you will surely find great restaurants that serve delicious food for you. Now, if you're a vegan, you may be wondering, all of this is good, but what if I'm looking for wonderful vegan cuisine to enjoy in Toronto? Will I be able to find something good? Will I as a student find good vegan restaurants near universities? Which neighborhoods can a company that specializes in vegan products promote its products? 

### 1.2. Business problem

Identify existing vegan restaurants in Toronto close to the city's universities, and identify neighborhoods where vegan companies can partner with restaurants to promote their products.

### 1.3. Audience

Vegan audience that comes to Toronto to study and companies interested in selling and promoting vegan products.

## 2. Data

### 2.1. Data description

For this analysis, we will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, the neighborhood data is not readily available on the internet. So, we will use a Wikipedia page that contains all the information we need to explore and cluster the neighborhoods in Toronto. The wikipedia page is available in https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.

We will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format that we can do our analysis.

We will use the Foursquare API to identify the existing vegan restaurants in Toronto and existing restaurants near universities with a distance up to 3000 meters. We will use a list of boroughs and neighborhoods of Toronto with their coordinates (latitude and longitude) to obtain the venues of each boroughs and neighborhoods of Toronto. The coordinates dataset is available in http://cocl.us/Geospatial_data.

### 2.2. Data cleaning

The data cleaning process consisted of obtaining postal codes for the city of Toronto from Wikipedia page. All borough and neighborhood with value "Not assigned" were removed from the dataframe.

In [29]:
import pandas as pd

# Create dataframe by using pandas and show first 10 lines
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df[0].head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [30]:
# We've assigned index 0 to a new Pandas dataframe
df_toronto = df[0]

# Drop cells with a borough that is Not assigned
index_NA = df_toronto[(df_toronto['Borough']=='Not assigned')].index
df_toronto.drop(index_NA, inplace=True)
df_toronto.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [31]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df_toronto.loc[df_toronto.Neighborhood=='Not assigned','Neighborhood'] = df_toronto.Neighborhood
df_toronto.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [32]:
df_toronto.shape

(103, 3)

Then we obtain the geographic data of the Toronto neighborhoods. We merged the two dataframe, creating a new one with all neighborhoods and their respective coordinates (latitude and longitude).

In [33]:
# We used Pandas to read csv file that has the geographical coordinates of each postal code to a dataframe
df_coordinators = pd.read_csv('http://cocl.us/Geospatial_data')
df_coordinators.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [34]:
# We merged the df_coordinators into df_toronto.
df_toronto = pd.merge(left=df_toronto,right=df_coordinators,on='Postal Code')
df_toronto.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [35]:
# Check numbers of Boroughs and Neighborhoods in the df_toronto
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_toronto['Borough'].unique()),
        df_toronto.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


We verify the geographic data collected by plotting the neighborhoods on the map.

In [36]:
import folium

# Toronto, Ontario
latitude = 43.6529
longitude = -79.3849

#Create map of Toronto using Latitude and Longitude values
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=11)

#add markers to map for neighborhoods
for lat,lng, borough, neighborhood in zip(df_toronto['Latitude'],df_toronto['Longitude'],df_toronto['Borough'],df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
    
map_toronto

### 2.3. Checking Foursquare API data

We invoke the Foursquare API to check the returned data. We researched all existing venues around the University of Toronto at a distance of up to 1000 meters.

In [37]:
# University of Toronto, Toronto, Ontario
neighborhood_latitude = 43.72149
neighborhood_longitude = -79.37881

In [38]:
hide_me

# Foursquare credencials
CLIENT_ID = '<id>' # your Foursquare ID
CLIENT_SECRET = '<secret>' # your Foursquare Secret
VERSION = '20180604'

# URL
LIMIT=100
radius=1000
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    neighborhood_latitude,
    neighborhood_longitude,
    radius,
    LIMIT
)

In [ ]:
import requests # library to handle requests

results = requests.get(url).json()
results

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

I've cleaned the returned json and I've structured it into a pandas dataframe.

In [ ]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

# First 10 rows
nearby_venues.head(10)

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

## 3. Methodology

## 4. Results

## 5. Discussion

## 6. Conclusion

## 7. References

https://bigseventravel.com/2020/02/best-vegan-restaurants-in-the-world-2020/

https://wtvox.com/sustainable-living/2019-the-world-of-vegan-but-how-many-vegans-are-in-the-world/

https://www.dailyhawker.ca/vegan-restaurants-in-toronto/

https://coursera.org
